## Imports and Functions

In [91]:
import pandas as pd
import numpy as np
import json
import os


def garmincoladjust(df):
    """
    Function renames Garmin column headers
    INPUT: garmin dataframe
    OUTPUT: garmin dataframe with new column headers
    """
    df[['Date', 'Clock Time']] = df['Date'].str.split(expand=True)
    colnamedict = {'Avg Vertical Oscillation':'Avg Vertical Oscillation (cm)', 
                   'Avg Ground Contact Time':'Avg GC Time (ms)',
                   'Avg Vertical Ratio':'Avg Vertical Ratio (%)', 'Avg Stride Length':'Avg Stride Length (m)',
                   'Elev Gain':'Elev Gain (ft)', 'Elev Loss':'Elev Loss (ft)'}
    df = df.rename(columns=colnamedict)
    df.Date = pd.to_datetime(df.Date)
    return df

def garminDate(date):
    dlist = date.split('/')
    month = dlist[0].zfill(2)
    day = dlist[1].zfill(2)
    year = dlist[2]
    newdate = year + '-' + month + '-' + day
    return newdate


def secs2hmm(sec):
    """
    Function takes time in seconds and converts to string in 'H:MM'
    INPUT: integer value of seconds
    OUTPUT: string in the form of 'H:MM'
    """
    totalminutes = sec / 60.0
    totalhours = int(totalminutes / 60)
    remainminutes = int(totalminutes % 60)
    hmm = str(totalhours) + ':' + str(remainminutes).zfill(2)
    return hmm


def mmss2sec(mmss):
    """
    Function takes time in 'MM:SS' and converts it to seconds.
    INPUT: string time 'MM:SS'
    OUTPUT: integer value of total seconds
    """
    lst = mmss.split(':')
    minuteint = int(lst[0])
    secondsint = int(lst[1])
    totalsecs = minuteint*60 + secondsint
    return totalsecs


def hmm2sec(hmm):
    """
    Function takes time in H:MM and converts it to integer value of seconds.
    INPUT: string time in 'H:MM'
    OUTPUT: integer value of total seconds
    """
    lst = hmm.split(':')
    hourint = int(lst[0])
    minint = int(lst[1])
    totalsecs = hourint*3600 + minint*60
    return totalsecs


def hhmmss2sec(hhmmss):
    """
    Function takes time in HH:MM:SS and converts it to an integer value of seconds.
    INPUT: string time 'HH:MM:SS'
    OUTPUT: integer value of total seconds
    """
    lst = hhmmss.split(':')
    hourint = int(lst[0])
    minint = int(lst[1])
    secint = int(lst[2])
    totalsecs = hourint*3600 + minint*60 + secint
    return totalsecs


def ms2hmm(ms):
    """
    Function takes in an integer value. Then it converts that number 
    to a string in the form of H:MM to represent the hours and minutes.
    INPUT: number as string type
    OUTPUT: String in the form H:MM to represent hours and minutes
    """
    secs = float(ms) / 1000
    minutes = secs/60
    hours = int(minutes/60)
    remminutes = int(minutes%60)
    hmm = str(hours) + ':' + str(remminutes).zfill(2)
    return hmm


## Garmin Data

In [56]:
## Cell reads in two csv files of garmin data, one old with all accumulated data, the other with new data to add

# Read in old and new csv files
rundfold = pd.read_csv('clean_garmin_data_NEW.csv')
rundfnew = garmincoladjust(pd.read_csv('Activities.csv')).set_index('Date')


# Concat the two dataframes into an updated dataframe, drop duplicates, overwrite clean file
rundf = pd.concat([rundfold, rundfnew], ignore_index=True)
rundf = rundf.set_index('Date')
rundf = rundf.sort_values(by='Date')
rundf = rundf.drop_duplicates()
rundf.to_csv('clean_garmin_data_NEW.csv')
rundfold.to_csv('clean_garmin_data_OLD.csv')